<a href="https://colab.research.google.com/github/coldsober-irene/NLP-fellowship/blob/main/week1Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
import pandas as pd
import re
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# open file with pandas
file = "/content/drive/MyDrive/NLP fellowship/week1 tasks/Eng_Kin-Paralleldata (1).csv"
opened_file = pd.read_csv(file)
opened_file.head()

,ID,English,Kinyarwanda
0,ID_IFVTMXW8,F_all back to threading messages by subject,Inyuma Kuri Ubutumwa ku Ikivugwaho
1,ID_SBXUPXE3,You will have to restart GIMP for the followin...,Kuri Ongera utangire kugirango i Amahinduka Ku...
2,ID_ZOCZ3PXQ,The radio button whose group this widget belon...,Akabuto bya Itsinda iyi Kuri
3,ID_6AVBJ1YS,Print all subdirectories in a directory.,Byose Ububiko bwungirije in a bushyinguro
4,ID_FW4V44WK,Paste the contents of the clipboard as a new i...,i Ibigize Bya i Ububikokoporora Nka A Gishya I...


In [8]:
# dataFrame columns
cols = opened_file.columns
kinyarwanda = cols[2]
english = cols[1]

'Kinyarwanda'

In [ ]:
# get all english words
all_english = opened_file[english]
all_kinyarwanda = opened_file[kinyarwanda]
all_kinyarwanda

In [ ]:
# turn all text into lower case text
lowerCase_english = [line.lower() for line in opened_file[english]]
lowerCase_kinyarwanda = [line.lower() for line in opened_file[kinyarwanda]]


In [ ]:
# remove digit in text
english_withoutNumbers = [re.sub("\d+", "", line) for line in lowerCase_english]
kinyarwanda_withoutNumbers = [re.sub("\d+", "", line) for line in lowerCase_kinyarwanda]


In [ ]:
# remove panctuation
pattern = re.compile(r"[^\w\s]")
english_withoutPanct = [re.sub(pattern, "", line) for line in english_withoutNumbers]
kinyarwanda_withoutPanct = [re.sub(pattern, "", line) for line in kinyarwanda_withoutNumbers]


In [ ]:
# remove the white space
english_withoutSpace = [line.strip() for line in english_withoutPanct]
kinyarwanda_withoutSpace = [line.strip() for line in kinyarwanda_withoutPanct]
kinyarwanda_withoutSpace
